# 📘 Lecture: RNN Backpropagation Through Time (BPTT)

---

## 🎯 Objective

In this lecture, we continue our study of **Recurrent Neural Networks (RNNs)** by focusing on **Backpropagation Through Time (BPTT)** — the fundamental algorithm by which RNNs learn from sequential data. Specifically, we'll cover:

- A quick recap of **forward propagation** in simple RNNs.
- Understanding how RNNs are **unfolded through time** to visualize the computational graph.
- Detailed explanation of **weight updates** using backpropagation for different types of weights.
- Understanding the application of the **chain rule** and gradient flow in a recurrent setting.
- The role of **gradient descent** in minimizing the loss function for sequence learning.

---

## 🔁 Forward Propagation in a Simple RNN

Before understanding backpropagation, let’s briefly revisit the **forward pass**, where information flows from input to output.

### 1. **Basic Architecture & Unfolding in Time**

A simple RNN processes sequential data by taking an input $x_t$ at each time step $t$ and combining it with a hidden state $h_{t-1}$ from the previous time step to compute a new hidden state $h_t$. This hidden state can then be used to produce an output $y_t$. The recurrent nature means the same set of weights are reused across all time steps.

Let's consider a sequence with $T$ time steps (e.g., $T=3$ words in a sentence). The RNN "unfolds" over these time steps:

* **Input sequence:** $x_1, x_2, \dots, x_T$ (e.g., $x_{i1}, x_{i2}, x_{i3}$)
* **Hidden states:** $h_0$ (initial), $h_1, h_2, \dots, h_T$
* **Outputs (optional at each step):** $\hat{y}_1, \hat{y}_2, \dots, \hat{y}_T$
* **Final output (if only one prediction at the end):** $\hat{y}_T$

### 2. **Forward Computation at Each Time Step**

At each time step $t$ (from $t=1$ to $T$), the hidden state $h_t$ is computed. For the first step ($t=1$), $h_{t-1}$ is the initial hidden state $h_0$.

The computation involves multiplying inputs and previous hidden states by their respective weights, adding a bias, and then applying a non-linear activation function.

* **Hidden State Calculation:**
    $$h_t = \text{Activation}_h(W_{xh}x_t + W_{hh}h_{t-1} + b_h)$$
    * **$x_t$**: The input vector at time step $t$.
    * **$W_{xh}$**: The **input-to-hidden weight matrix**. It transforms the current input into the hidden state space.
    * **$h_{t-1}$**: The hidden state vector from the previous time step. For $t=1$, this is $h_0$, the initial hidden state.
    * **$W_{hh}$**: The **hidden-to-hidden (recurrent) weight matrix**. It captures the influence of the previous hidden state on the current one.
    * **$b_h$**: The **hidden layer bias vector**.
    * **$\text{Activation}_h(\cdot)$**: A non-linear activation function (e.g., $\tanh$, ReLU). The lecture mentions $\tanh$.

* **Output Calculation (if a prediction is made at each step, or at the final step $T$):**
    $$ \hat{y}_t = \text{Activation}_y(W_{hy}h_t + b_y) $$
    * **$\hat{y}_t$**: The predicted output vector at time step $t$.
    * **$W_{hy}$**: The **hidden-to-output weight matrix**. It transforms the current hidden state into the output space.
    * **$b_y$**: The **output layer bias vector**.
    * **$\text{Activation}_y(\cdot)$**: An activation function suitable for the output task (e.g., $\text{sigmoid}$ for binary classification, $\text{softmax}$ for multi-class classification). The lecture uses $\text{sigmoid}$ for the final output.

For the example given in the lecture with a final output $\hat{y}$ derived from $h_3$:
$$ \hat{y} = \text{sigmoid}(W_{hy}h_3 + b_y) $$

---

## 🔙 Backpropagation Through Time (BPTT)

After the forward pass, we calculate a **Loss Function** $\mathcal{L}$ that quantifies the difference between our network's predictions $\hat{y}$ (or $\hat{y}_t$) and the true target values $y$ (or $y_t$).

$$ \mathcal{L} = \text{Loss}(y, \hat{y}) $$

Our primary goal during training is to **minimize this loss**. This is achieved by iteratively adjusting the network's weights and biases ($W_{xh}, W_{hh}, W_{hy}, b_h, b_y$). We use **gradient descent** (or its variants) for this optimization.

The general weight update rule for gradient descent is:
$$ W_{\text{new}} = W_{\text{old}} - \eta \cdot \frac{\partial \mathcal{L}}{\partial W_{\text{old}}} $$
* **$W_{\text{new}}$**: The updated weight matrix.
* **$W_{\text{old}}$**: The current weight matrix before update.
* **$\eta$**: The **learning rate**, a small positive scalar hyperparameter that controls the step size of the update.
* **$\frac{\partial \mathcal{L}}{\partial W_{\text{old}}}$**: The **gradient** of the loss function with respect to the weight $W_{\text{old}}$. This term tells us the direction and magnitude to change $W_{\text{old}}$ to decrease the loss.

In RNNs, calculating these gradients involves **Backpropagation Through Time (BPTT)**, which is essentially applying the standard backpropagation algorithm to the unfolded computational graph of the RNN. Since weights like $W_{xh}$ and $W_{hh}$ are shared across all time steps, their gradients must be accumulated over time.

---

### 🧩 Step-by-Step Weight Updates

Let's detail how the gradients are calculated for each set of weights.

### 1. 🎯 Updating Output Weights ($W_{hy}$) and Bias ($b_y$)

The output weights and bias are typically only involved in the final step of generating a prediction from a hidden state. Assuming a loss is computed at the final time step $T$ based on $\hat{y}_T$:

* **Gradient with respect to $W_{hy}$:**
    The loss $\mathcal{L}$ depends on $\hat{y}_T$, and $\hat{y}_T$ directly depends on $W_{hy}$.
    Using the chain rule:
    $$ \frac{\partial \mathcal{L}}{\partial W_{hy}} = \frac{\partial \mathcal{L}}{\partial \hat{y}_T} \cdot \frac{\partial \hat{y}_T}{\partial W_{hy}} $$
    * $\frac{\partial \mathcal{L}}{\partial \hat{y}_T}$: How much the loss changes with respect to a change in the final predicted output. This depends on the specific loss function used.
    * $\frac{\partial \hat{y}_T}{\partial W_{hy}}$: How much the final predicted output changes with respect to a change in $W_{hy}$. This involves the derivative of the output activation function $\text{Activation}_y'(\cdot)$ and the hidden state $h_T$.

* **Update Rule for $W_{hy}$:**
    $$ W_{hy}^{\text{new}} = W_{hy}^{\text{old}} - \eta \cdot \frac{\partial \mathcal{L}}{\partial W_{hy}} $$

* **Similarly for $b_y$:**
    $$ \frac{\partial \mathcal{L}}{\partial b_y} = \frac{\partial \mathcal{L}}{\partial \hat{y}_T} \cdot \frac{\partial \hat{y}_T}{\partial b_y} $$
    $$ b_y^{\text{new}} = b_y^{\text{old}} - \eta \cdot \frac{\partial \mathcal{L}}{\partial b_y} $$

These gradients are relatively straightforward as they do not involve propagating the error signal backward through multiple recurrent time steps.

---

### 2. 🔁 Updating Recurrent (Hidden-to-Hidden) Weights ($W_{hh}$) and Bias ($b_h$)

The recurrent weight $W_{hh}$ and bias $b_h$ are crucial because they are reused at **every single time step** ($t = 1, \dots, T$) to connect hidden states. Therefore, their gradients must account for their influence on the loss through all time steps. This involves summing the gradients from each point in time where they were used.

* **Total Gradient for $W_{hh}$:**
    $$ \frac{\partial \mathcal{L}}{\partial W_{hh}} = \sum_{k=1}^{T} \frac{\partial \mathcal{L}}{\partial W_{hh} \text{ at time } k} $$
    Here, $\frac{\partial \mathcal{L}}{\partial W_{hh} \text{ at time } k}$ represents the specific contribution of $W_{hh}$ at time step $k$ to the total loss.

* **Chain Rule for a single contribution (e.g., $W_{hh}$ at time $k$ affecting the final loss at $T$):**
    The path of influence goes: $W_{hh} \to h_k \to h_{k+1} \to \dots \to h_T \to \hat{y}_T \to \mathcal{L}$.
    This translates to a product of derivatives:
    $$ \frac{\partial \mathcal{L}}{\partial W_{hh} \text{ at time } k} = \frac{\partial \mathcal{L}}{\partial \hat{y}_T} \cdot \frac{\partial \hat{y}_T}{\partial h_T} \cdot \frac{\partial h_T}{\partial h_{T-1}} \cdot \ldots \cdot \frac{\partial h_{k+1}}{\partial h_k} \cdot \frac{\partial h_k}{\partial W_{hh}} $$
    * **$\frac{\partial h_j}{\partial h_{j-1}}$**: This is the critical recurrent term. As derived in the previous section (and will be expanded below), it is typically $\text{Activation}_h'(z_j) \cdot W_{hh}$.
    * **$\frac{\partial h_k}{\partial W_{hh}}$**: How the hidden state at time $k$ changes with respect to $W_{hh}$. This term depends on $h_{k-1}$ and $\text{Activation}_h'(\cdot)$.

* **Update Rule for $W_{hh}$:**
    $$ W_{hh}^{\text{new}} = W_{hh}^{\text{old}} - \eta \cdot \frac{\partial \mathcal{L}}{\partial W_{hh}} $$

* **Similarly for $b_h$:**
    $$ \frac{\partial \mathcal{L}}{\partial b_h} = \sum_{k=1}^{T} \frac{\partial \mathcal{L}}{\partial b_h \text{ at time } k} $$
    The chain rule for each contribution is similar to $W_{hh}$ but ends with $\frac{\partial h_k}{\partial b_h}$.
    $$ b_h^{\text{new}} = b_h^{\text{old}} - \eta \cdot \frac{\partial \mathcal{L}}{\partial b_h} $$

---

### 3. 📥 Updating Input Weights ($W_{xh}$)

The input weights $W_{xh}$ are also shared and used at each time step to incorporate the current input $x_t$ into the hidden state $h_t$. Their gradients must also be accumulated over time.

* **Total Gradient for $W_{xh}$:**
    $$ \frac{\partial \mathcal{L}}{\partial W_{xh}} = \sum_{k=1}^{T} \frac{\partial \mathcal{L}}{\partial W_{xh} \text{ at time } k} $$

* **Chain Rule for a single contribution (e.g., $W_{xh}$ at time $k$ affecting the final loss at $T$):**
    The path of influence goes: $W_{xh} \to h_k \to h_{k+1} \to \dots \to h_T \to \hat{y}_T \to \mathcal{L}$.
    $$ \frac{\partial \mathcal{L}}{\partial W_{xh} \text{ at time } k} = \frac{\partial \mathcal{L}}{\partial \hat{y}_T} \cdot \frac{\partial \hat{y}_T}{\partial h_T} \cdot \frac{\partial h_T}{\partial h_{T-1}} \cdot \ldots \cdot \frac{\partial h_{k+1}}{\partial h_k} \cdot \frac{\partial h_k}{\partial W_{xh}} $$
    * **$\frac{\partial h_k}{\partial W_{xh}}$**: How the hidden state at time $k$ changes with respect to $W_{xh}$. This term depends on $x_k$ and $\text{Activation}_h'(\cdot)$.

* **Update Rule for $W_{xh}$:**
    $$ W_{xh}^{\text{new}} = W_{xh}^{\text{old}} - \eta \cdot \frac{\partial \mathcal{L}}{\partial W_{xh}} $$

---

## 🔍 Decomposing the Recurrent Gradient Term: $\frac{\partial h_j}{\partial h_{j-1}}$

This term is the cornerstone of understanding how gradients flow (and potentially vanish or explode) in RNNs. It represents how sensitive the hidden state at time $j$ is to the hidden state at time $j-1$.

Recall the hidden state calculation:
$$ h_j = \text{Activation}_h(W_{xh}x_j + W_{hh}h_{j-1} + b_h) $$

Let $z_j = W_{xh}x_j + W_{hh}h_{j-1} + b_h$ be the net input to the activation function at time $j$.

Using the chain rule:
$$ \frac{\partial h_j}{\partial h_{j-1}} = \frac{d \text{Activation}_h(z_j)}{d z_j} \cdot \frac{\partial z_j}{\partial h_{j-1}} $$

* **Derivative of the Activation Function:** $\frac{d \text{Activation}_h(z_j)}{d z_j}$ is the derivative of the activation function, evaluated at $z_j$. For $\tanh(z)$, the derivative is $1 - \tanh^2(z)$. For $\text{sigmoid}(z)$, the derivative is $\text{sigmoid}(z)(1 - \text{sigmoid}(z))$.

* **Derivative of the Net Input:** $\frac{\partial z_j}{\partial h_{j-1}}$:
    $$ \frac{\partial}{\partial h_{j-1}} (W_{xh}x_j + W_{hh}h_{j-1} + b_h) $$
    Since $W_{xh}x_j$ and $b_h$ are constant with respect to $h_{j-1}$, their derivatives are zero. The derivative of $W_{hh}h_{j-1}$ with respect to $h_{j-1}$ is $W_{hh}$.

Therefore, the crucial recurrent term is:
$$ \frac{\partial h_j}{\partial h_{j-1}} = \text{Activation}_h'(z_j) \cdot W_{hh} $$
Where $\text{Activation}_h'(z_j)$ is the element-wise derivative of the activation function.

This is the term that gets multiplied repeatedly in the gradient calculation for earlier time steps. If its magnitude is consistently less than 1 (as is often the case with sigmoid and tanh for inputs far from zero, and small $W_{hh}$ values), the gradient will **vanish**. If its magnitude is consistently greater than 1, the gradient will **explode**.

---

## 📉 Gradient Descent & Convergence

The entire process of BPTT, calculating these complex gradients, and then applying the weight update rules using $\text{gradient descent}$ (or its variants like Adam, RMSprop) is performed iteratively over many training epochs.

The ultimate goal of this iterative optimization process is to:
* **Minimize the loss function**: By moving in the direction opposite to the gradient, we steadily decrease the error.
* **Reach a global minimum (or a good local minimum)**: This signifies that the network's weights have been optimized to represent the underlying patterns in the sequential data as accurately as possible.
* **Enable the RNN to learn accurate predictions**: A minimized loss indicates that the RNN has effectively learned the long-term and short-term dependencies necessary to make precise outputs.

---

## ❓ Why Do We Need BPTT?

BPTT is indispensable for training RNNs for several key reasons:

* **Sequential Learning**: RNNs are specifically designed for sequential data where the order and historical context matter. BPTT provides the mechanism for the model to learn these temporal dependencies by propagating error signals across time.
* **Shared Parameters**: Unlike feedforward networks, RNNs reuse the same set of recurrent weights ($W_{hh}$) and input weights ($W_{xh}$) at every time step. BPTT correctly accumulates the gradients from all these instances of usage to ensure effective updates.
* **Memory in Learning**: The hidden state acts as the RNN's "memory." BPTT allows the error signal to flow back into this memory mechanism, enabling the model to adjust how it retains and forgets information from previous steps in a principled and optimized way.
* **Effective Optimization**: Without BPTT, an RNN would not be able to adjust its weights to minimize errors that depend on the entire sequence. It would be akin to training a feedforward network without backpropagation.

---

## ✅ Conclusion

In this lecture, we've taken a deep dive into **Backpropagation Through Time (BPTT)**, the core learning algorithm for simple Recurrent Neural Networks. We've:

* Reviewed the computational steps of **forward propagation** in an unfolded RNN.
* Understood how **gradients are calculated** for the output weights ($W_{hy}$), recurrent weights ($W_{hh}$), and input weights ($W_{xh}$) using the **chain rule**, accounting for their shared nature across time.
* Examined the crucial recurrent derivative term $\frac{\partial h_j}{\partial h_{j-1}}$ which explains the **vanishing (and exploding) gradient problems** in simple RNNs.
* Reinforced the role of **gradient descent** in iteratively adjusting these weights to minimize the loss and enable effective sequence learning.

> 🔍 BPTT is critical in making RNNs truly learn from sequential patterns. However, its very mechanism (repeated multiplication of derivatives) also reveals the inherent limitations of simple RNNs, particularly the vanishing gradient problem. This understanding forms the foundation for appreciating the advancements brought by more sophisticated architectures like LSTMs and GRUs, which were specifically designed to overcome these challenges and enable learning of truly long-term dependencies.

---